<a href="https://colab.research.google.com/github/MarijkeBeersmans/MiddelNER-Middelnederlandse-named-entity-recognition/blob/main/ModelMiddelNederlandseNER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Middelnederlandse NER

## prep

In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Embedding, Bidirectional, Input
from tensorflow.keras.models import Model

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import f1_score, classification_report
print(tf.__version__)

#to get reproducible results
tf.random.set_seed(1234)
np.random.seed(1234)

import spacy
print(spacy.__version__) #we need version 3 or higher
# !pip install --upgrade spacy
# !pip install transformers
# !pip install seqeval
# !pip install datasets

2.8.0
3.3.0


In [3]:
!git clone https://github.com/MarijkeBeersmans/MiddelNER-Middelnederlandse-named-entity-recognition.git

Cloning into 'MiddelNER-Middelnederlandse-named-entity-recognition'...
remote: Enumerating objects: 79, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 79 (delta 28), reused 61 (delta 20), pack-reused 0
Unpacking objects: 100% (79/79), done.


In [4]:
%cd /content/MiddelNER-Middelnederlandse-named-entity-recognition

/content/MiddelNER-Middelnederlandse-named-entity-recognition


In [5]:
path = 'C14NL-PoS-annotated/TRAIN.tsv'
TRAIN_DF = pd.read_csv(path, sep="\t", encoding = "UTF-8", header=None)

path2 = 'C14NL-PoS-annotated/TEST.tsv'
TEST_DF = pd.read_csv(path2, sep="\t", encoding = "UTF-8", header=None)

path3 = 'C14NL-PoS-annotated/DEV.tsv'
DEV_DF = pd.read_csv(path3, sep="\t", encoding = "UTF-8", header=None)

from string import punctuation

def clean_words(word):
  word = word.casefold()
  word = word.translate(str.maketrans('', '', punctuation))
  return word

# print(TRAIN[0])
TRAIN_DF[0] = TRAIN_DF[0].apply(clean_words)
DEV_DF[0] = DEV_DF[0].apply(clean_words)
TEST_DF[0] = TEST_DF[0].apply(clean_words)

TRAIN_DF.head()

,0,1,2,3,4
0,wi,O,365,O098p33302,1
1,hebben,O,365,O098p33302,1
2,ij,B-MONEY,365,O098p33302,1
3,sc,I-MONEY,365,O098p33302,1
4,par,I-MONEY,365,O098p33302,1


## enkel LSTM, goede dataset

In [6]:
def split(list_a, chunk_size):

  for i in range(0, len(list_a), chunk_size):
    yield list_a[i:i + chunk_size]

def map_sentence(s):
  doc = [(w, t) for w, t in zip(s[0].values.tolist(),
                                      s[1].values.tolist())]
  chunks = split(doc, 250)
  return list(chunks)
     
def get_sent(df):
  grouped = df.groupby(2).apply(map_sentence)
  sentences = [chunk for s in grouped for chunk in s]
  return sentences

In [7]:
TRAIN = get_sent(TRAIN_DF)
TEST = get_sent(TEST_DF)
DEV = get_sent(DEV_DF)

TRAIN[0]

[('wi', 'O'),
 ('hebben', 'O'),
 ('ij', 'B-MONEY'),
 ('sc', 'I-MONEY'),
 ('par', 'I-MONEY'),
 ('vte', 'O'),
 ('den', 'O'),
 ('ghemene', 'O'),
 ('dachwande', 'O'),
 ('ter', 'O'),
 ('doot', 'O'),
 ('e', 'O'),
 ('door', 'O'),
 ('moet', 'O'),
 ('derfachteghede', 'O'),
 ('vzoeke', 'O'),
 ('e', 'O'),
 ('gheuen', 'O'),
 ('onsen', 'O'),
 ('meyere', 'O'),
 ('vi', 'B-MONEY'),
 ('d', 'I-MONEY'),
 ('et', 'O'),
 ('elken', 'O'),
 ('van', 'O'),
 ('seuen', 'O'),
 ('ghesuoren', 'O'),
 ('laten', 'O'),
 ('die', 'O'),
 ('wi', 'O'),
 ('daer', 'O'),
 ('hebben', 'O'),
 ('i', 'B-MONEY'),
 ('d', 'I-MONEY'),
 ('en', 'O'),
 ('maecht', 'O'),
 ('een', 'O'),
 ('mensche', 'O'),
 ('vzoeken', 'O'),
 ('ouer', 'O'),
 ('al', 'O'),
 ('door', 'O'),
 ('maer', 'O'),
 ('de', 'O'),
 ('meyer', 'O'),
 ('e', 'O'),
 ('de', 'O'),
 ('late', 'O'),
 ('vors', 'O'),
 ('moeten', 'O'),
 ('haer', 'O'),
 ('recht', 'O'),
 ('hebben', 'O'),
 ('van', 'O'),
 ('elken', 'O'),
 ('dier', 'O'),
 ('an', 'O'),
 ('marchiert', 'O'),
 ('hoe', 'O'),
 ('dat

In [8]:
uniq_words = set(list(TRAIN_DF[0])+list(TEST_DF[0])+list(DEV_DF[0])) 

uniq_words = list(uniq_words)
uniq_tags = list(TRAIN_DF[1].unique())

# Dictionary word to index
word2idx = {w : i + 2 for i, w in enumerate(uniq_words)}
word2idx["UNK"] = 1
word2idx["PAD"] = 0

# Dictionary label to index 
tag2idx = {t : i + 1 for i, t in enumerate(uniq_tags)}
tag2idx["PAD"] = 0

idx2word = {i: w for w, i in word2idx.items()}
idx2tag = {i: w for w, i in tag2idx.items()}

In [9]:
len(word2idx), len(uniq_words), len(tag2idx), len(uniq_tags), len(idx2word), len(idx2tag)

(5500, 5498, 10, 9, 5500, 10)

In [10]:
idx2word[15], word2idx["jaer"], tag2idx['B-PERS'], idx2tag[5]

('eruelikere', 5321, 4, 'I-PERS')

In [11]:
X_train = [[w[0] for w in s] for s in TRAIN]
y_train = [[w[1] for w in s] for s in TRAIN]


X_test = [[w[0] for w in s] for s in TEST]
y_test = [[w[1] for w in s] for s in TEST]

X_dev = [[w[0] for w in s] for s in DEV]
y_dev = [[w[1] for w in s] for s in DEV]

In [12]:
max_len = max([len(s) for s in TRAIN])
# X_train

X_train_tokens = [[word2idx[w] for w in s] for s in X_train]
y_train_cat = [[tag2idx[w] for w in s] for s in y_train]
y_train_cat = [to_categorical(i, num_classes = len(tag2idx)) for i in y_train_cat]


# Let's limit or pad each sequence to have sentences of the same length
X_train_tokens= pad_sequences(maxlen = max_len, sequences = X_train_tokens, padding = "post", value = word2idx["PAD"])
y_train_cat = pad_sequences(maxlen = max_len, sequences = y_train_cat, padding = "post", value = tag2idx["PAD"])


X_test_tokens = [[word2idx[w] for w in s] for s in X_test]
y_test_cat = [[tag2idx[w] for w in s] for s in y_test]
y_test_cat = [to_categorical(i, num_classes = len(tag2idx)) for i in y_test_cat]


# Let's limit or pad each sequence to have sentences of the same length
X_test_tokens= pad_sequences(maxlen = max_len, sequences = X_test_tokens, padding = "post", value = word2idx["PAD"])
y_test_cat = pad_sequences(maxlen = max_len, sequences = y_test_cat, padding = "post", value = tag2idx["PAD"])

X_dev_tokens = [[word2idx[w] for w in s] for s in X_dev]
y_dev_cat = [[tag2idx[w] for w in s] for s in y_dev]
y_dev_cat = [to_categorical(i, num_classes = len(tag2idx)) for i in y_dev_cat]

# Let's limit or pad each sequence to have sentences of the same length
X_dev_tokens= pad_sequences(maxlen = max_len, sequences = X_dev_tokens, padding = "post", value = word2idx["PAD"])
y_dev_cat = pad_sequences(maxlen = max_len, sequences = y_dev_cat, padding = "post", value = tag2idx["PAD"])

In [13]:
# Number of data points passed in each iteration
batch_size = 64 
# Passes through entire dataset
epochs = 100
# Dimension of embedding vector
embedding_size = 100
# number of 
n_units = 100
# learning rate
lr = 0.001
# validation split
val_split=0.1

num_words = len(word2idx)

In [14]:
from sklearn.utils.extmath import softmax
num_words = len(word2idx)

import tensorflow.keras as keras

early_stopping = tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)

model = keras.Sequential()
model.add(keras.layers.Embedding(num_words, embedding_size, input_length=max_len, mask_zero=True))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(embedding_size, return_sequences=True,)))
model.add(keras.layers.Dense(len(tag2idx), activation='softmax', name='output'))

model.summary() # explain params
optimizer = keras.optimizers.Adam(learning_rate=lr)

model.compile(optimizer=optimizer,
                loss='categorical_crossentropy',
                metrics=['accuracy'])

model.fit(X_train_tokens, y_train_cat, batch_size=batch_size, epochs=epochs,
          validation_data=(X_dev_tokens, y_dev_cat), callbacks=[early_stopping])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 250, 100)          550000    
                                                                 
 bidirectional (Bidirectiona  (None, 250, 200)         160800    
 l)                                                              
                                                                 
 output (Dense)              (None, 250, 10)           2010      
                                                                 
Total params: 712,810
Trainable params: 712,810
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
4/4 [==============================] - 18s 1s/step - loss: 1.6383 - accuracy: 0.6095 - val_loss: 1.6681 - val_accuracy: 0.8512
Epoch 2/100
4/4 [==============================] - 1s 152ms/step - loss: 1.4999 - accuracy: 0.8374 - val_loss

In [15]:
from sklearn import metrics

def flat_classification_report(y_true, y_pred):
  y_true_report = []
  y_pred_report = []

  for item in y_true:
    y_true_report+= item

  for item in y_pred:
    y_pred_report+= item

  y_pred_report = [item if item != 'PAD' else 'O' for item in y_pred_report]
 
  return metrics.classification_report(y_true_report, y_pred_report)

def predict_test(my_model): 
  y_pred = my_model.predict(X_test_tokens)
  y_pred = np.argmax(y_pred, axis=-1)
  y_pred = [[idx2tag[i] for i in row] for row in y_pred]
  y_pred = [y_seq[:len(y_test[i])] for i, y_seq in enumerate(y_pred)]
  return y_pred


y_pred = predict_test(model)
print(flat_classification_report(y_pred=y_pred, y_true=y_test))

              precision    recall  f1-score   support

      B-DATE       0.97      0.75      0.85        40
       B-LOC       0.91      0.56      0.70        85
     B-MONEY       0.64      0.72      0.68        53
      B-PERS       0.93      0.79      0.86       254
      I-DATE       1.00      0.66      0.79       151
       I-LOC       0.25      0.10      0.14        10
     I-MONEY       0.81      0.92      0.86       123
      I-PERS       0.81      0.63      0.71       224
           O       0.95      0.98      0.97      4310

    accuracy                           0.94      5250
   macro avg       0.81      0.68      0.73      5250
weighted avg       0.94      0.94      0.93      5250



In [16]:
flat_X_test = [item for lst in X_test for item in lst]
flat_y_test = [item for lst in y_test for item in lst]
flat_y_pred = [item for lst in y_pred for item in lst]

df_errors = pd.DataFrame(list(zip(flat_X_test, flat_y_test, flat_y_pred)), columns=['word', 'ent', 'prediction'])

In [53]:
df_errors.head(20)

df_errors[df_errors['ent'] != df_errors['prediction']]

df_errors[10:5000]

,word,ent,prediction
10,sinte,O,B-LOC
11,clare,O,O
12,typre,B-LOC,B-LOC
13,kenne,O,O
14,dat,O,O
...,...,...,...
4995,gheerart,B-PERS,B-PERS
4996,wauel,I-PERS,I-PERS
4997,alse,O,O
4998,schepenne,O,O


## With BERT

In [24]:
# !pip install transformers
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer

In [25]:
TRAIN_DF.head()
DEV_DF.head()
TEST_DF.head()

,0,1,2,3,4
0,wie,O,688,H072p40001,140
1,suster,O,688,H072p40001,140
2,lisebette,B-PERS,688,H072p40001,140
3,bi,O,688,H072p40001,140
4,den,O,688,H072p40001,140


In [26]:
def split(list_a, chunk_size):

  for i in range(0, len(list_a), chunk_size):
    yield list_a[i:i + chunk_size]

def map_sentence(s):
  doc = [[w, t] for w, t in zip(s[0].values.tolist(),
                                      s[1].values.tolist())]
  chunks = split(doc, 100)
  return list(chunks)
     
def get_sent(df):
  grouped = df.groupby(2).apply(map_sentence)
  sentences = [chunk for s in grouped for chunk in s]
  return sentences

In [27]:
def chunk_df(df):
  counter = 0
  new_df = pd.DataFrame(columns=['token', 'tag', 'num'])

  sentences = get_sent(df)

  for chunk in sentences:
    chunk = pd.DataFrame(chunk, columns=['token', 'tag'])
    chunk['num'] = counter
    counter = counter + 1
    new_df = new_df.append(chunk)

  return new_df

In [29]:
import json

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
TRAIN_DF[1] = le.fit_transform(TRAIN_DF[1])
for row in TRAIN_DF[1]:
  assert type(row) == int
DEV_DF[1] = le.transform(DEV_DF[1])
for row in DEV_DF[1]:
  assert type(row) == int
TEST_DF[1] = le.transform(TEST_DF[1])
for row in TEST_DF[1]:
  assert type(row) == int
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_name_mapping)

def get_dataset(df):
  for_BERT = []
  
  df = chunk_df(df)
  df_GROUPED = df.groupby('num')
  for group in df_GROUPED.groups:
    group = df_GROUPED.get_group(group)
    tokens = list(group['token'])
    tags = list(group['tag'])
    for element in tags:
      assert type(element) == int
    dct = {'tokens': tokens, 'tags': tags}
    for_BERT.append(json.dumps(dct, ensure_ascii=False))

  return for_BERT

TRAIN_json = get_dataset(TRAIN_DF)

print(TRAIN_json)

import os

os.makedirs("data/Middelner", exist_ok=True)
with open("data/Middelner/train.json", "w+") as f:
    f.write("\n".join(TRAIN_json))


{'B-DATE': 0, 'B-LOC': 1, 'B-MONEY': 2, 'B-PERS': 3, 'I-DATE': 4, 'I-LOC': 5, 'I-MONEY': 6, 'I-PERS': 7, 'O': 8}
['{"tokens": ["wi", "hebben", "ij", "sc", "par", "vte", "den", "ghemene", "dachwande", "ter", "doot", "e", "door", "moet", "derfachteghede", "vzoeke", "e", "gheuen", "onsen", "meyere", "vi", "d", "et", "elken", "van", "seuen", "ghesuoren", "laten", "die", "wi", "daer", "hebben", "i", "d", "en", "maecht", "een", "mensche", "vzoeken", "ouer", "al", "door", "maer", "de", "meyer", "e", "de", "late", "vors", "moeten", "haer", "recht", "hebben", "van", "elken", "dier", "an", "marchiert", "hoe", "dat", "ne", "waer", "een", "vzoect", "dit", "was", "wetteleke", "vclareert", "bi", "maninghen", "va", "boidin", "keyacke", "te", "dien", "tiden", "onse", "meyere", "e", "bi", "vonnesse", "va", "zeuen", "ghezuornen", "laten", "e", "bi", "consente", "va", "der", "peuzen", "zuste", "e", "broede", "va", "den", "spet", "e", "ooc", "bi"], "tags": [8, 8, 2, 6, 6, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 

In [30]:
!tail data/Middelner/train.json

{"tokens": ["van", "ziere", "zielen", "ziner", "vader", "ende", "moeder", "ziner", "vrienden", "ende", "allen", "anderen", "zielen", "daer", "af", "dat", "hi", "yet", "te", "bet", "heeft", "ghehadt", "in", "sinen", "leuene", "heeft", "ghekent", "ende", "ghelijt", "kent", "ende", "lijt", "dat", "hi", "heeft", "ghegheuen", "teeweliken", "daghen", "voert", "meer", "ende", "gheeft", "als", "nv", "bi", "der", "tenueren", "van", "desen", "wetteliken", "brieuen", "puerlijc", "om", "gode", "ende", "in", "puerer", "aermoesenen", "fransen", "van", "der", "beke", "clerc", "vten", "bisscopdomme", "van", "doernike", "als", "nv", "capellaen", "zijnde", "van", "der", "vors", "capellen", "ter", "vors", "capelrien", "bouf", "ende", "anders", "niet", "ende", "in", "vermeereringhen", "ende", "verbeteringhe", "van", "der", "vors", "capelrien", "ende", "van", "den", "dienste", "ende", "loue", "van", "gode", "een"], "tags": [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,

In [31]:
TEST_json = get_dataset(TEST_DF)

os.makedirs("data/Middelner", exist_ok=True)
with open("data/Middelner/test.json", "w+") as f:
    f.write("\n".join(TEST_json))

DEV_json = get_dataset(DEV_DF)

os.makedirs("data/Middelner", exist_ok=True)
with open("data/Middelner/validation.json", "w+") as f:
    f.write("\n".join(DEV_json))

In [32]:
!tail data/Middelner/validation.json

{"tokens": ["vriheden", "altoes", "behauden", "lijsbetten", "ende", "callen", "vors", "hare", "vors", "xl", "s", "par", "siaers", "de", "welke", "xl", "sc", "par", "siaers", "erfeleker", "renten", "mathijs", "vors", "heeft", "belooft", "bewijst", "en", "beset", "op", "de", "vors", "erue", "en", "haboet", "alsoe", "voren", "verclaert", "jn", "kenness", "van", "wareden", "so", "hebben", "wy", "franss", "crupenninc", "franss", "van", "der", "crayen", "en", "jan", "jacops", "als", "schepn", "in", "audende", "in", "desen", "tijt", "dese", "pnte", "cyrogphie", "ghezeghelt", "elc", "onser", "met", "zinen", "ppren", "zeghele", "vte", "hanghende", "dit", "was", "ghedaen", "jnt", "jaer", "xiijciiijxx", "en", "xix", "den", "xxviijste", "dach", "van", "septembre"], "tags": [8, 8, 8, 3, 8, 3, 8, 8, 8, 2, 6, 6, 8, 8, 8, 2, 6, 6, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 3, 7, 3, 7, 7, 7, 8, 3, 7, 8, 8, 8, 1, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,

In [33]:
# !pip install datasets

from datasets import load_dataset

dataset = load_dataset("data/Middelner")

Using custom data configuration Middelner-d58d2a48fd04cc98


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/Middelner-d58d2a48fd04cc98/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [34]:
dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 463
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 61
    })
    validation: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 71
    })
})

In [35]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("GroNLP/bert-base-dutch-cased")

Downloading:   0%|          | 0.00/254 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/236k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [36]:
label_names = list(le_name_mapping.values())
# print(label_names)
# for label in label_names:
#   print(type(label))

In [37]:
#Get the values for input_ids, token_type_ids, attention_mask
def tokenize_adjust_labels(all_samples_per_split):
  tokenized_samples = tokenizer.batch_encode_plus(all_samples_per_split["tokens"], is_split_into_words=True, truncation=True, max_length=512, add_special_tokens = True)
  #tokenized_samples is not a datasets object so this alone won't work with Trainer API, hence map is used 
  #so the new keys [input_ids, labels (after adjustment)]
  #can be added to the datasets dict for each train test validation split
  total_adjusted_labels = []
  print(len(tokenized_samples["input_ids"]))
  for k in range(0, len(tokenized_samples["input_ids"])):
    prev_wid = -1
    word_ids_list = tokenized_samples.word_ids(batch_index=k)
    existing_label_ids = all_samples_per_split["tags"][k]
    i = -1
    adjusted_label_ids = []
   
    for wid in word_ids_list:
      if(wid is None):
        adjusted_label_ids.append(-100)
      elif(wid!=prev_wid):
        i = i + 1
        adjusted_label_ids.append(existing_label_ids[i])
        prev_wid = wid
      else:
        label_name = label_names[existing_label_ids[i]]
        adjusted_label_ids.append(existing_label_ids[i])
        
    total_adjusted_labels.append(adjusted_label_ids)
  tokenized_samples["labels"] = total_adjusted_labels
  return tokenized_samples

tokenized_dataset = dataset.map(tokenize_adjust_labels, batched=True)


  0%|          | 0/1 [00:00<?, ?ba/s]

463


  0%|          | 0/1 [00:00<?, ?ba/s]

61


  0%|          | 0/1 [00:00<?, ?ba/s]

71


In [38]:
# tokenized_dataset['train'][1]

In [39]:
# !pip install seqeval

from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
import numpy as np
from datasets import load_metric

metric = load_metric("seqeval")
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    flattened_results = {
        "overall_precision": results["overall_precision"],
        "overall_recall": results["overall_recall"],
        "overall_f1": results["overall_f1"],
        "overall_accuracy": results["overall_accuracy"],
    }
    for k in results.keys():
      if(k not in flattened_results.keys()):
        flattened_results[k+"_f1"]=results[k]["f1"]

    return flattened_results

In [40]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
# !pip install wandb

In [41]:
tokenizer.mask_token_id

4

In [42]:

model = AutoModelForTokenClassification.from_pretrained("GroNLP/bert-base-dutch-cased", num_labels=len(label_names))
training_args = TrainingArguments(
    output_dir="./fine_tune_bert_output",
    evaluation_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=24,
    weight_decay=0.01,
    logging_steps = 1000,
    report_to="tensorboard",
    run_name = "ep_10_tokenized_11",
    save_strategy='no',
    warmup_ratio = 0.1 
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

Downloading:   0%|          | 0.00/417M [00:00<?, ?B/s]

Some weights of the model checkpoint at GroNLP/bert-base-dutch-cased were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at GroNLP/bert-base-dutch-cased

Step,Training Loss,Validation Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=696, training_loss=0.1786038080851237, metrics={'train_runtime': 905.2797, 'train_samples_per_second': 12.275, 'train_steps_per_second': 0.769, 'total_flos': 1410529858228674.0, 'train_loss': 0.1786038080851237, 'epoch': 24.0})

In [43]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tags, tokens. If tags, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 71
  Batch size = 16


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 8 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 3 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 7 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 1 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 2 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 6 seem

{'epoch': 24.0,
 'eval_loss': 0.23290450870990753,
 'eval_overall_accuracy': 0.9589421390999415,
 'eval_overall_f1': 0.0,
 'eval_overall_precision': 0.0,
 'eval_overall_recall': 0.0,
 'eval_runtime': 2.6604,
 'eval_samples_per_second': 26.688,
 'eval_steps_per_second': 1.879}

In [46]:
from transformers import pipeline
model.to('cpu')
nlp = pipeline('ner', model=model, tokenizer=tokenizer)
s = "kennesse van hondert schellinghen par en daer dit ghedaen was wetteleke in al der manieren vorscreuen daer was ouer arent blancstrein"
print(nlp(s))

[{'entity': 'LABEL_8', 'score': 0.9994753, 'index': 1, 'word': 'ken', 'start': 0, 'end': 3}, {'entity': 'LABEL_8', 'score': 0.99970645, 'index': 2, 'word': '##ness', 'start': 3, 'end': 7}, {'entity': 'LABEL_8', 'score': 0.99974614, 'index': 3, 'word': '##e', 'start': 7, 'end': 8}, {'entity': 'LABEL_8', 'score': 0.99817085, 'index': 4, 'word': 'van', 'start': 9, 'end': 12}, {'entity': 'LABEL_2', 'score': 0.5264579, 'index': 5, 'word': 'hond', 'start': 13, 'end': 17}, {'entity': 'LABEL_3', 'score': 0.33280694, 'index': 6, 'word': '##er', 'start': 17, 'end': 19}, {'entity': 'LABEL_3', 'score': 0.41460624, 'index': 7, 'word': '##t', 'start': 19, 'end': 20}, {'entity': 'LABEL_2', 'score': 0.54592866, 'index': 8, 'word': 'schel', 'start': 21, 'end': 26}, {'entity': 'LABEL_6', 'score': 0.9207681, 'index': 9, 'word': '##ling', 'start': 26, 'end': 30}, {'entity': 'LABEL_6', 'score': 0.88262165, 'index': 10, 'word': '##hen', 'start': 30, 'end': 33}, {'entity': 'LABEL_6', 'score': 0.9332512, 'ind

## With SpaCy

In [18]:
# install spacy version adapted to cuda 9.2 and transformers
#after that you must probably restart the runtime
!pip install -U pip setuptools wheel
!pip install -U spacy[cuda92,transformers,lookups]

     |████████████████████████████████| 2.1 MB 5.3 MB/s 
     |████████████████████████████████| 1.1 MB 40.6 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.0/55.0 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 KB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 62.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.18.0
    Uninstalling transformers-4.18.0:
      Successfully uninstalled transformers-4.18.0


In [19]:
#change the torch and torchivison version to working in cuda 9.2
!pip install torch==1.7.1+cu92 torchvision==0.8.2+cu92 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.3/577.3 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 22.7 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.12.0+cu113
    Uninstalling torchvision-0.12.0+cu113:
      Successfully uninstalled torchvision-0.12.0+cu113
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.11.0+cu113
    Uninstalling torchaudio-0.11.0+cu113:
      Successfully uninstalled torchaudio-0.11.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of 

In [20]:
#add the path to cuda libraries
!export CUDA_PATH="/usr/local/cuda-9.2"
!export LD_LIBRARY_PATH=$CUDA_PATH/lib64:$LD_LIBRARY_PATH

!curl https://colab.chainer.org/install | sh -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1580  100  1580    0     0   5622      0 --:--:-- --:--:-- --:--:--  5642
+ apt -y -q install cuda-libraries-dev-10-0
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-10-0 is already the newest version (10.0.130-1).
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.
+ pip install -q cupy-cuda100  chainer 
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
+ set +ex
Installation succeeded!


In [21]:
!python -m spacy train -g 0 /content/MiddelNER-Middelnederlandse-named-entity-recognition/C14NL-PoS-annotated/config.cfg -o /content/

ℹ Saving to output directory: /content
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-05-04 07:36:14,375] [INFO] Set up nlp object from config
[2022-05-04 07:36:14,410] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-05-04 07:36:14,421] [INFO] Created vocabulary
[2022-05-04 07:36:14,423] [INFO] Finished initializing nlp object
[2022-05-04 07:37:10,778] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     75.33    0.00    0.00    0.00    0.00
  1     200       5264.77  10523.44   43.21   50.16   37.96    0.43
  3     400        841.82   2382.05   76.83   77.02   76.64    0.77
  4     600        311.20   1168.56   

In [22]:
#Generating prediction on the test dataset
from spacy.training import offsets_to_biluo_tags

f=open("C14NL-PoS-annotated/test_for_spacy.txt", "r")
test=f.read()
test=test.split("\n\n")[:-1]
print(len(test))


from spacy.training import offsets_to_biluo_tags
from spacy.tokens import Doc

class WhitespaceTokenizer:
    def __init__(self, vocab):
        self.vocab = vocab

    def __call__(self, text):
        words = text.split(" ")
        spaces = [True] * len(words)
        # Avoid zero-length tokens
        for i, word in enumerate(words):
            if word == "":
                words[i] = " "
                spaces[i] = False
        # Remove the final trailing space
        if words[-1] == " ":
            words = words[0:-1]
            spaces = spaces[0:-1]
        else:
           spaces[-1] = False
            
        return Doc(self.vocab, words=words, spaces=spaces)

import spacy

nlp = spacy.load("/content/model-best")
nlp.tokenizer = WhitespaceTokenizer(nlp.vocab)

# print(test.split("\t"))

true=[[x.split("\t")[1] for x in text.split("\n") ] for text in test]
true=[y for x in true for y in x]
token=[[x.split("\t")[0] for x in text.split("\n") ] for text in test]
token=[y for x in token for y in x]

predictions=[[(ent.start_char, ent.end_char, ent.label_) for ent in nlp(" ".join([x.split("\t")[0] for x in text.split("\n")])).ents] for text in test]
predictions=[offsets_to_biluo_tags(nlp(" ".join([x.split("\t")[0] for x in text.split("\n")])), predictions[i]) for i,text in enumerate(test)]


predictions=[y.replace("U-", "B-").replace("L-", "I-") for x in predictions for y in x]

16


In [23]:
from sklearn.metrics import classification_report

report=classification_report(true, predictions)

print(report)

              precision    recall  f1-score   support

      B-DATE       0.86      0.78      0.82        40
       B-LOC       0.86      0.67      0.75        85
     B-MONEY       0.89      0.91      0.90        53
      B-PERS       0.94      0.82      0.87       254
      I-DATE       0.81      0.85      0.83       151
       I-LOC       1.00      0.20      0.33        10
     I-MONEY       0.84      0.92      0.88       123
      I-PERS       0.94      0.75      0.84       224
           O       0.96      0.98      0.97      4310

    accuracy                           0.95      5250
   macro avg       0.90      0.76      0.80      5250
weighted avg       0.95      0.95      0.95      5250

